In [ ]:
from aust_covid.model import build_model
from autumn.infrastructure.remote import springboard
import arviz as az
import numpy as np

from emutools.calibration import plot_priors, tabulate_priors
from inputs.constants import INPUTS_PATH, SUPPLEMENT_PATH
from aust_covid.calibration import get_priors, get_all_priors, get_targets
from emutools.tex import DummyTexDoc, StandardTexDoc
from emutools.parameters import load_param_info
from aust_covid.utils import add_image_to_doc
from aust_covid.inputs import get_ifrs
from aust_covid.plotting import plot_targets, plot_dispersion_examples

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Supplementary material to analysis of Australia's 2022 COVID-19 epidemic", 'austcovid')

In [ ]:
param_info = load_param_info()
all_priors = get_all_priors()
get_priors(False, param_info['abbreviations'], app_doc)
priors_table = tabulate_priors(all_priors, param_info)
app_doc.include_table(priors_table, section='Priors', col_splits=[0.25] * 4)
fig = plot_priors(all_priors, param_info['abbreviations'], 4, 100, 0.1, 0.99)
caption = 'Illustrations of prior distributions implemented in calibration algorithm.'
add_image_to_doc(fig, 'prior_distributions', caption, app_doc, 'Priors')

In [ ]:
targets = get_targets(app_doc)
fig = plot_targets(targets)
add_image_to_doc(fig, 'target_fig', 'Calibration targets with raw data from which they were derived.', app_doc, 'Targets')

In [ ]:
epi_model = build_model(DummyTexDoc())

In [ ]:
rts = springboard.task.RemoteTaskStore()
rts.cd('projects/aust_covid/alternate_analyses')
mt = rts.get_managed_task('2023-10-09T1251-none-d20k-t10k-b5k')
# mt.download_all()
idata = az.from_netcdf(mt.local / 'output/calib_full_out.nc')

In [ ]:
prior_names = [p.name for p in get_priors(False, param_info['abbreviations'], DummyTexDoc()) if p.name != 'imm_prop']
parameters = param_info['value'].to_dict()
parameters.update(get_ifrs(DummyTexDoc()))

In [ ]:
colours = {'notifications_ma': '10, 10, 100', 'deaths_ma': '100, 10, 10'}
centiles = np.linspace(0.1, 0.9, 9)
fig = plot_dispersion_examples(idata, epi_model, parameters, prior_names, targets, colours, centiles)
fig.update_layout(showlegend=False)
caption = 'Examples of the effect of values of the negative binomial distribution dispersion parameter.'
add_image_to_doc(fig, 'dispersion_examples', caption, app_doc, 'Targets')

In [ ]:
app_doc.write_doc()